In [2]:
from IPython.display import display_html
from sqlalchemy import create_engine
import pandas as pd
import warnings
warnings.simplefilter("ignore")

host='localhost'
database='eicu'
username='purav'
port = "5432"

postgres_str =('postgresql://'+username+':root'+'@'+host+':'+port+'/'+database)
#   Create the connection
cnx = create_engine(postgres_str)

In [3]:
# pd.read_sql_query('''SELECT * FROM eicuu.patient limit 10;''', cnx)

In [4]:
patient = pd.read_sql('SELECT * FROM eicuu.patient', cnx)
#diagnosis = pd.read_sql('SELECT * FROM diagnosis', con)
apache = pd.read_sql('SELECT * FROM eicuu.apacheapsvar', cnx)

result1 = pd.merge(patient,
                 apache,
                 on="patientunitstayid", 
                 how='inner')

OperationalError: (psycopg2.OperationalError) FATAL:  role "purav" does not exist
 (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
result=result1.drop(['patienthealthsystemstayid', 'ethnicity','wardid','apacheadmissiondx','admissionheight','hospitaladmittime24',
        'hospitaladmitoffset','hospitaladmitsource', 'hospitaldischargeyear',
       'hospitaldischargetime24', 'hospitaldischargeoffset',
       'hospitaldischargelocation','unittype',
       'unitadmittime24', 'unitadmitsource', 'unitvisitnumber', 'unitstaytype',
       'admissionweight', 'dischargeweight', 'unitdischargetime24',
       'unitdischargeoffset', 'unitdischargelocation', 'unitdischargestatus',
       'uniquepid','apacheapsvarid', 'intubated','dialysis', 'eyes',
       'motor', 'verbal', 'meds','ph', 'hematocrit',
       'creatinine', 'albumin','pco2','glucose'],axis = 1)
print(result.columns)

In [6]:
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score

X_temp = result

X_temp = X_temp[X_temp.age != '> 89']
X_temp = X_temp[X_temp.age != '']

X_temp = X_temp[X_temp.hospitaldischargestatus != '']

X_temp = X_temp.astype({"age": np.int})
X_temp = X_temp[X_temp.age >= 18]

gender = pd.DataFrame(X_temp.gender.map({'Male': 1, 'Female': 0}))
hds = pd.DataFrame(X_temp.hospitaldischargestatus.map({'Alive': 1, 'Expired': 0}))
# print(hds.head())

X_temp['gender'] = gender['gender']
X_temp['hospitaldischargestatus'] = hds['hospitaldischargestatus']

X_temp=X_temp.dropna()

X_temp = X_temp.astype({"gender": np.int})


X_alive = X_temp.loc[X_temp['hospitaldischargestatus'] == 1]
X_expired = X_temp.loc[X_temp['hospitaldischargestatus'] == 0]

X_alive_restricted = X_alive.head(15000)

X_temp = X_alive_restricted.append(X_expired)

print(X.shape)

X = pd.DataFrame(X_temp.drop(['patientunitstayid'],axis=1))
Y = pd.DataFrame(X_temp['hospitaldischargestatus'])                 
X = pd.DataFrame(X.drop(['hospitaldischargestatus'],axis=1))

# # print(X.head(10))

Y = Y[Y.hospitaldischargestatus != '']
# Y = pd.DataFrame(Y['hospitaldischargestatus'].map({'Alive': 1, 'Expired': 0}))



# df = Y.groupby('hospitaldischargestatus').size()
# print(df)

# print(type(X))
# age = np.asarray(X['age'])
# a = []
# for i in age:
#     try:
#         if( int(i)):
#             a.append(i)
#     except:
#         print("skipping")
    
    
print(X.shape,Y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42, stratify=Y)

log_regressor_lasso   = LogisticRegression(penalty='l1', max_iter = 1e8)
log_regressor_lasso.fit(X_train, y_train)

lasso_pred = log_regressor_lasso.predict(X_test)
lasso_f1_score=(f1_score(y_test,lasso_pred))

print(lasso_f1_score)

#     Age
#     Heart Rate
#     Systolic Blood Pressure
#     Temperature
#     Glasgow Coma Scale
#     Mechanical Ventilation or CPAP
#     PaO2
#     FiO2
#     Urine Output
#     Blood Urea Nitrogen
#     Sodium
#     Potassium
#     Bicarbonate
#     Bilirubin
#     White Blood Cell
#     Chronic diseases
#     Type of admission

NameError: name 'result' is not defined

0.9529393278730383
0.9529196865240704